In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI
import json
import re

class TableRAG:
    def __init__(self, embedding_model='all-MiniLM-L6-v2', openai_api_key=None):
        """
        Initialize TableRAG with embedding model and optional OpenAI client
        
        Args:
            embedding_model (str): Sentence transformer model
            openai_api_key (str): OpenAI API key for generation
        """
        # Embedding model for table semantic representation
        self.embedding_model = SentenceTransformer(embedding_model)
        
        # OpenAI client for generation (optional)
        if openai_api_key:
            self.openai_client = OpenAI(api_key=openai_api_key)
        else:
            self.openai_client = None

    def parse_table(self, table_data, source_type='dataframe'):
        """
        Parse table from various sources
        
        Args:
            table_data: Input table data
            source_type (str): Type of input source
        
        Returns:
            pd.DataFrame: Parsed and standardized DataFrame
        """
        if source_type == 'dataframe':
            return table_data
        elif source_type == 'csv':
            return pd.read_csv(table_data)
        elif source_type == 'excel':
            return pd.read_excel(table_data)
        elif source_type == 'json':
            return pd.DataFrame(json.loads(table_data))
        else:
            raise ValueError("Unsupported source type")

    def generate_table_embeddings(self, dataframe):
        """
        Generate semantic embeddings for table rows
        
        Args:
            dataframe (pd.DataFrame): Input DataFrame
        
        Returns:
            dict: Embeddings for each row
        """
        # Convert each row to a textual representation
        row_texts = dataframe.apply(
            lambda row: ' | '.join(row.astype(str)), 
            axis=1
        )
        
        # Generate embeddings
        embeddings = self.embedding_model.encode(row_texts.tolist())
        
        return {
            'embeddings': embeddings,
            'row_texts': row_texts
        }

    def semantic_table_search(self, dataframe, query, top_k=3):
        """
        Perform semantic search across table rows
        
        Args:
            dataframe (pd.DataFrame): Input DataFrame
            query (str): Search query
            top_k (int): Number of top results to return
        
        Returns:
            pd.DataFrame: Top matching rows
        """
        # Generate table embeddings
        table_embeddings = self.generate_table_embeddings(dataframe)
        
        # Embed query
        query_embedding = self.embedding_model.encode([query])[0]
        
        # Compute cosine similarity
        similarities = cosine_similarity([query_embedding], table_embeddings['embeddings'])[0]
        
        # Get top-k indices
        top_indices = similarities.argsort()[-top_k:][::-1]
        
        # Return top matching rows with similarity scores
        results = dataframe.iloc[top_indices].copy()
        results['similarity_score'] = similarities[top_indices]
        
        return results

    def generate_narrative(self, query, retrieved_rows):
        """
        Generate narrative explanation using retrieved rows
        
        Args:
            query (str): Original query
            retrieved_rows (pd.DataFrame): Retrieved matching rows
        
        Returns:
            str: Generated narrative explanation
        """
        if self.openai_client is None:
            raise ValueError("OpenAI API key not provided")
        
        # Convert retrieved rows to string representation
        context = retrieved_rows.to_string(index=False)
        
        # Construct prompt
        prompt = f"""
        Context: {context}
        
        Query: {query}
        
        Based on the provided context, generate a comprehensive and 
        insightful narrative that directly addresses the query. 
        Explain the key findings and provide relevant insights.
        """
        
        # Generate response
        response = self.openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an expert data analyst"},
                {"role": "user", "content": prompt}
            ]
        )
        
        return response.choices[0].message.content

    def analyze_table_statistics(self, dataframe):
        """
        Provide comprehensive statistical analysis of the table
        
        Args:
            dataframe (pd.DataFrame): Input DataFrame
        
        Returns:
            dict: Statistical summary
        """
        # Numeric column analysis
        numeric_columns = dataframe.select_dtypes(include=[np.number]).columns
        numeric_stats = dataframe[numeric_columns].agg([
            'mean', 'median', 'min', 'max', 'std'
        ]).to_dict()
        
        # Categorical column analysis
        categorical_columns = dataframe.select_dtypes(include=['object']).columns
        categorical_stats = {
            col: dataframe[col].value_counts(normalize=True).to_dict()
            for col in categorical_columns
        }
        
        return {
            'numeric_stats': numeric_stats,
            'categorical_stats': categorical_stats
        }

def main():
    # Example usage with medical indications data
    medical_data = pd.DataFrame([
        ['Inflammatory Conditions', 147.2, 'High-cost'],
        ['Cancer', 47.5, 'Critical'],
        ['Migraine Headaches', 22.7, 'Moderate'],
        ['Heart Diseases', 20.9, 'High-risk'],
        ['Diabetes', 19.2, 'Chronic']
    ], columns=['Indication', 'Total Cost (M$)', 'Risk Category'])

    # Initialize TableRAG (replace with your OpenAI API key)
    table_rag = TableRAG(openai_api_key='')

    # Perform semantic search
    query = "What are high-cost medical conditions?"
    search_results = table_rag.semantic_table_search(
        medical_data, 
        query, 
        top_k=2
    )
    print("Semantic Search Results:\n", search_results)

    # Generate narrative explanation
    narrative = table_rag.generate_narrative(query, search_results)
    print("\nNarrative Explanation:\n", narrative)

    # Perform statistical analysis
    stats = table_rag.analyze_table_statistics(medical_data)
    print("\nTable Statistics:\n", json.dumps(stats, indent=2))

if __name__ == "__main__":
    main()

Semantic Search Results:
                 Indication  Total Cost (M$) Risk Category  similarity_score
0  Inflammatory Conditions            147.2     High-cost          0.575739
3           Heart Diseases             20.9     High-risk          0.398201

Narrative Explanation:
 Based on the provided data, the high-cost medical condition identified is "Inflammatory Conditions," with a total cost of $147.2 million. This condition falls under the high-cost category due to the substantial financial resources required for its treatment and management. The similarity score of 0.575739 indicates a relatively high correlation with this classification.

On the other hand, "Heart Diseases" is classified as high-risk with a total cost of $20.9 million and a similarity score of 0.398201. While heart diseases are characterized as high-risk, they may not necessarily incur as high costs as inflammatory conditions, as indicated by the stark contrast in total cost between the two conditions.

These fin